# SageMaker Serial Inference Pipeline(SIP) with Scikit-learn and Linear Learner
Typically a Machine Learning (ML) process consists of few steps: data gathering with various ETL jobs, pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm. 
In many cases, when the trained model is used for processing real time or batch prediction requests, the model receives data in a format which needs to pre-processed (e.g. featurized) before it can be passed to the algorithm. In the following notebook, we will demonstrate how to deploy a Pipeline (Data preprocessing and Linear Learner) as an Inference Pipeline behind a single Endpoint for real time inference. The Pipeline is made up a Scikit-learn Preprocessor and a learner model pretrained on the Abalone dataset to guess the age of Abalone with physical features. 
The dataset is available from [UCI Machine Learning](https://archive.ics.uci.edu/ml/datasets/abalone).

### Table of contents
* [Inference Pipeline with Scikit preprocessor and Linear Learner](#inference_pipeline)
 * [Set up the inference pipeline](#pipeline_setup)
 * [Make a request to our pipeline endpoint](#pipeline_inference_request)
 * [Delete Endpoint](#delete_endpoint)

Let's first create our Sagemaker session and role, and create a S3 prefix to use for the notebook example.

## Serial Inference Pipeline with Scikit preprocessor and Linear Learner <a class="anchor" id="serial_inference"></a>


### Step 1: Set up the inference pipeline <a class="anchor" id="pipeline_setup"></a>
Setting up a Machine Learning pipeline can be done with the Pipeline Model. This sets up a list of models in a single endpoint; in this example, we configure our pipeline model with the fitted Scikit-learn inference model and the fitted Linear Learner model. Deploying the model follows the same ```deploy``` pattern in the SDK.

In [1]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
sagemaker_session = sagemaker.Session()
aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = "Scikit-LinearLearner-pipeline-abalone-example"

!aws s3 cp scikit-learn/model.tar.gz s3://{bucket}/{prefix}/scikit-learn/model.tar.gz
!aws s3 cp linear-learner/model.tar.gz s3://{bucket}/{prefix}/linear-learner/model.tar.gz

import sagemaker 
image_uri = sagemaker.image_uris.retrieve("linear-learner", "us-east-1")
    
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.model import Model
scikit_learn_inference_model = SKLearnModel(
            model_data=f"s3://{bucket}/{prefix}/scikit-learn/model.tar.gz",
            entry_point="scikit-learn/sklearn_abalone_featurizer.py",
            framework_version="1.0-1",
            role=aws_role,
            sagemaker_session=sess
            )

linear_learner_model = Model(
            model_data=f"s3://{bucket}/{prefix}/linear-learner/model.tar.gz",
            image_uri=image_uri,
            role=aws_role,
            sagemaker_session=sess
)

upload: scikit-learn/model.tar.gz to s3://sagemaker-us-east-1-171503325295/Scikit-LinearLearner-pipeline-abalone-example/scikit-learn/model.tar.gz
upload: linear-learner/model.tar.gz to s3://sagemaker-us-east-1-171503325295/Scikit-LinearLearner-pipeline-abalone-example/linear-learner/model.tar.gz


In [2]:
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_name = "inference-pipeline-" + timestamp_prefix
endpoint_name = "inference-pipeline-inference-pipeline" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=aws_role, models=[scikit_learn_inference_model, linear_learner_model]
)

sm_model.deploy(initial_instance_count=1, instance_type="ml.c4.xlarge", endpoint_name=endpoint_name)

-----------------!

### Step 2: Make a request to our pipeline endpoint <a class="anchor" id="pipeline_inference_request"></a>

Here we just grab the first line from the test data (you'll notice that the inference python script is very particular about the ordering of the inference request data). The ```ContentType``` field configures the first container, while the ```Accept``` field configures the last container. You can also specify each container's ```Accept``` and ```ContentType``` values using environment variables.

We make our request with the payload in ```'text/csv'``` format, since that is what our script currently supports. If other formats need to be supported, this would have to be added to the ```output_fn()``` method in our entry point. Note that we set the ```Accept``` to ```application/json```, since Linear Learner does not support ```text/csv``` ```Accept```. The prediction output in this case is trying to guess the number of rings the abalone specimen would have given its other physical features; the actual number of rings is 10.

In [3]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

payload = "M, 0.44, 0.365, 0.125, 0.516, 0.2155, 0.114, 0.155"
actual_rings = 10
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session, serializer=CSVSerializer()
)

print(predictor.predict(payload))

b'{"predictions": [{"score": 9.528051376342773}]}'


### Step 3: Delete Endpoint <a class="anchor" id="delete_endpoint"></a>
Once we are finished with the endpoint, we clean up the resources!

In [4]:
# sm_client = sagemaker_session.boto_session.client("sagemaker")
# predictor.delete_model()
# sm_client.delete_endpoint(EndpointName=endpoint_name)